In [33]:
import pandas as pd
import geopandas as gpd
import altair as alt
from vega_datasets import data
import json

In [36]:
### read geopackage
shape = '../data/regional_smooth_wgs84.gpkg'
regions = gpd.read_file(shape)
regions.drop(columns=['cir_sena', 'Region'], inplace=True)

regions

,codregion,geometry
0,15,"MULTIPOLYGON (((-69.41531 -17.65521, -69.33660..."
1,1,"MULTIPOLYGON (((-70.16034 -20.21070, -70.16020..."
2,2,"MULTIPOLYGON (((-70.70164 -25.89174, -70.70059..."
3,12,"MULTIPOLYGON (((-67.32026 -55.92015, -67.28998..."
4,11,"MULTIPOLYGON (((-75.57992 -48.75893, -75.58039..."
5,3,"MULTIPOLYGON (((-71.26109 -28.49257, -71.26109..."
6,4,"MULTIPOLYGON (((-71.53526 -32.16410, -71.53534..."
7,5,"MULTIPOLYGON (((-80.77333 -33.72415, -80.75527..."
8,13,"MULTIPOLYGON (((-70.73098 -32.92615, -70.70736..."
9,10,"MULTIPOLYGON (((-74.84555 -43.59982, -74.84631..."


In [37]:
### read data from minsal
URL = 'https://www.minsal.cl/nuevo-coronavirus-2019-ncov/casos-confirmados-en-chile-covid-19/'

minsal = pd.read_html(URL)[0].iloc[2:]
minsal.columns = minsal.iloc[0]
minsal = minsal.iloc[1:-1]

### remove useless column
minsal.drop(columns=['% Casos totales**'], inplace=True)

### add region to merge with geopandas
minsal['codregion'] = [15, 1, 2, 3, 4, 5, 13, 6, 7, 16, 8, 9, 14, 10, 11, 12]

### de-format numbers
for col in ['Casos nuevos', 'Casos totales', 'Fallecidos']:
    minsal[col] = minsal[col].str.replace('\.', '').astype(int)
    
minsal

2,Región,Casos nuevos,Casos totales,Fallecidos,codregion
3,Arica y Parinacota,0,6,0,15
4,Tarapacá,2,10,0,1
5,Antofagasta,0,35,0,2
6,Atacama,0,2,0,3
7,Coquimbo,1,28,0,4
8,Valparaíso,7,115,1,5
9,Metropolitana,125,1420,4,13
10,O’Higgins,2,23,0,6
11,Maule,8,62,1,7
12,Ñuble,16,245,0,16


In [38]:
### merge
regions = regions.merge(minsal, on='codregion')

regions

,codregion,geometry,Región,Casos nuevos,Casos totales,Fallecidos
0,15,"MULTIPOLYGON (((-69.41531 -17.65521, -69.33660...",Arica y Parinacota,0,6,0
1,1,"MULTIPOLYGON (((-70.16034 -20.21070, -70.16020...",Tarapacá,2,10,0
2,2,"MULTIPOLYGON (((-70.70164 -25.89174, -70.70059...",Antofagasta,0,35,0
3,12,"MULTIPOLYGON (((-67.32026 -55.92015, -67.28998...",Magallanes,24,63,0
4,11,"MULTIPOLYGON (((-75.57992 -48.75893, -75.58039...",Aysén,0,2,0
5,3,"MULTIPOLYGON (((-71.26109 -28.49257, -71.26109...",Atacama,0,2,0
6,4,"MULTIPOLYGON (((-71.53526 -32.16410, -71.53534...",Coquimbo,1,28,0
7,5,"MULTIPOLYGON (((-80.77333 -33.72415, -80.75527...",Valparaíso,7,115,1
8,13,"MULTIPOLYGON (((-70.73098 -32.92615, -70.70736...",Metropolitana,125,1420,4
9,10,"MULTIPOLYGON (((-74.84555 -43.59982, -74.84631...",Los Lagos,23,151,0


In [39]:
### transform to json and to altair data (?!)
choro_json = json.loads(regions.to_json())
choro_data = alt.Data(values=choro_json['features'])

In [55]:
alt.Chart(choro_data).mark_geoshape(
    stroke='black',
    strokeWidth=1
).encode(
    color = 'properties.Casos totales:Q',
    tooltip = ['properties.Región:N', 'properties.Casos totales:Q']
).properties(
    width=800,
    height=800,
)

alt.Chart(...)